# Geração do gráfico

## Importando modulos necessários

In [2]:
from tensorflow import keras
from keras import layers

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

## Dados de entrada

In [3]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    x = layers.Conv2D(64, 3, strides=2, padding="same")(inputs)
    x = layers.Flatten()(x)
    x = layers.Dense(32)(x)


    outputs = layers.Dense(1, activation=None)(x)
    return keras.Model(inputs, outputs)



epochs = 20
image_size = (256,500,3)

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "TODO/PATH",
    validation_split=0.3,
    label_mode="binary",
    subset="both",
    seed=1,
    image_size=image_size[0:2],
)

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

model = make_model(input_shape=image_size, num_classes=2)

callbacks = [
    keras.callbacks.ModelCheckpoint("checkpoints/save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

model.save('assets/final_model.keras')

In [ ]:
train_ds.class_names

In [ ]:
model = keras.models.load_model('assetsfinal_model.keras')

img = keras.utils.load_img("assets/CP14.1AMPLITUDE.png", target_size=image_size)
plt.imshow(img)

img_array = keras.utils.img_to_array(img)
img_array = keras.ops.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = float(keras.ops.sigmoid(predictions[0][0]))
print(f"Esse espectograma representa {100 * (1 - score):.2f}% furo e {100 * score:.2f}% conforme.")